In [2]:
!pip install --quiet mistralai weaviate-client 2>/dev/null

In [3]:
import os
import re

import weaviate
from weaviate.classes.init import Auth
from weaviate.classes.config import Property, DataType

from mistralai import Mistral

In [4]:
api_key = "HCOHSJGVjzxKYjhBjO4u8rGmZFKObnLf"
embedding_model = "mistral-embed"

mistral_client = Mistral(api_key=api_key)

def get_embeddings(questions):
    response = mistral_client.embeddings.create(
        model=embedding_model,
        inputs=questions
    )
    return response.data 

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [5]:
url = "https://pgwkmycmtbor78al14g5ya.c0.asia-southeast1.gcp.weaviate.cloud"
key = "1TZymK1LSn6FnBTnvou7T5ghjJAXx2EItPgJ"

weaviate_client = weaviate.connect_to_weaviate_cloud(
    cluster_url=url,  
    auth_credentials=Auth.api_key(key)  
)

if weaviate_client.is_ready():
    print("✅ Successfully connected to Weaviate Cloud!")
else:
    print("❌ Failed to connect. Check your URL and API Key.")

✅ Successfully connected to Weaviate Cloud!


In [6]:
collection_name = "Llmexam"

if collection_name not in weaviate_client.collections.list_all():
    weaviate_client.collections.create(
        name=collection_name,
        properties=[
            Property(name="question_text", data_type=DataType.TEXT),
            Property(name="question_embedding", data_type=DataType.NUMBER_ARRAY),
        ],
        vectorizer_config=None 
    )
    print(f"✅ Collection '{collection_name}' created.")
else:
    print(f"⚡ Collection '{collection_name}' already exists.")

⚡ Collection 'Llmexam' already exists.


In [7]:
def store_questions(questions, embeddings):
    for question, emb in zip(questions, embeddings):
        vector = emb.embedding if hasattr(emb, "embedding") else emb
        llmexam.data.insert(
            properties={
                "question_text": question,
                "question_embedding": vector
            },
            vector=vector  
        )
    print(f"✅ {len(questions)} questions stored successfully!")

In [8]:
llmexam = weaviate_client.collections.get("llmexam")

questions = [
    "What is the capital of France?",
    "Who wrote 'To Kill a Mockingbird'?",
    "What is the square root of 64?",
    "What is the chemical symbol for water?",
    "Who painted the Mona Lisa?"
]

embeddings = get_embeddings(questions)
store_questions(questions, embeddings)

print("✅ Successfully stored questions in Weaviate.")

✅ 5 questions stored successfully!
✅ Successfully stored questions in Weaviate.
